In [10]:
def clean(census_data, race_data):
    import warnings
    import numpy as np
    # clean census data
    warnings. filterwarnings('ignore')

    # Only keep important columns
    census_data = census_data[['location','label','estimate','moe']]
    census_data['label'] = census_data['label'].str.replace('Estimate', '').str.replace('Total', '').str.replace('!!', '').str.replace(':', '')
    census_data['location'] = census_data['location'].str.replace('; St. Louis city; Missouri','')

    # Filter out aggregate columns
    all_totals = census_data[census_data['label']=='']
    total_tract_pops = np.array(all_totals['estimate'].tolist())
    tract_pop_proportions = total_tract_pops / np.sum(total_tract_pops)
    trans_totals_mask = ~census_data['label'].astype(str).str.contains('[$]')
    trans_totals = census_data.loc[trans_totals_mask][1:]
    income_totals = census_data[census_data['label'].str.startswith("$")]
    not_to_include = pd.concat([trans_totals, income_totals, all_totals])
    merged_df = pd.merge(census_data, not_to_include, how='left', indicator=True)
    no_totals = merged_df[merged_df['_merge'] == 'left_only'].drop(columns=['_merge'])[1:]
    split_data = no_totals['label'].str.split('$', 1, expand=True)
    no_totals[['transportation', 'income']] = split_data
    no_totals.drop(columns=['label'], inplace=True)
    no_totals['income'] = '$' + no_totals['income']
    from_this = no_totals['income'].unique()
    to_this = [1, 10000, 15000, 25000, 35000, 50000, 65000, 75000]
    no_totals['avg income'] = no_totals['income'].replace(from_this, to_this)
    clean_census_data = no_totals
    
    # clean race data
    # Only keep important columns
    race_data = race_data[['location','label','concept','estimate','moe']]
    race_data['label'] = race_data['label'].str.replace('Estimate', '').str.replace('Total', '').str.replace('!!', '').str.replace(':', '')
    race_data['location'] = race_data['location'].str.replace('; St. Louis city; Missouri','')
    race_data['concept'] = race_data['concept'].str.replace('MEANS OF TRANSPORTATION TO WORK ', '').str.replace(')','').str.replace('(','').str.replace('ALONE','')

    # Filter out aggregate rows and rename columns
    race_data = race_data[race_data['label'] != '']
    race_data = race_data.rename(columns={'label': 'transportation', 'concept': 'race'})
    
    # Group by census tract and mode of transportation
    by_tract_trans = clean_census_data.groupby(['location','transportation'])['estimate'].sum()

    # Find percentage of people who take public transport in each tract
    tract_pops = clean_census_data.groupby('location', as_index = False)['estimate'].sum()['estimate'].tolist()
    public_transit_pops = clean_census_data[clean_census_data['transportation']=='Public transportation (excluding taxicab)'].groupby(['location','transportation'], as_index = False)['estimate'].sum()['estimate'].tolist()
    total_public_transit_pops = sum(public_transit_pops)

    # Add percentages back in to groupby
    by_tract = clean_census_data.groupby('location', as_index = False)['estimate'].sum()
    by_tract['transit weight'] = np.array(public_transit_pops) / total_public_transit_pops
    by_tract['transit weight (pop)'] = np.divide(by_tract['transit weight'],tract_pop_proportions)
    
    # Group by census tract and income
    by_tract_inc = clean_census_data.groupby(['location','avg income'])['estimate'].sum()

    # Find average income of each tract
    tract_incomes = (clean_census_data['avg income']*clean_census_data['estimate']).groupby(clean_census_data['location']).sum().tolist()
    tract_totals = sum(tract_incomes)
    tract_incomes_new = np.array([tract_totals - i for i in tract_incomes])
    income_weight = tract_incomes_new / sum(tract_incomes_new)

    # Add percentages back in to groupby
    by_tract['income weight'] = income_weight
    by_tract['income weight (pop)'] = np.divide(by_tract['income weight'],tract_pop_proportions)
    
    # Find minority (nonwhite) population of each tract
    total_pops = race_data.groupby('location')['estimate'].sum().tolist()
    nonwhite_pops = race_data[race_data['race'] != 'WHITE '].groupby('location')['estimate'].sum().tolist()
    total_nonwhite_pop = sum(nonwhite_pops)

    # add back in to groupby
    by_tract['race weight'] = np.array(nonwhite_pops) / total_nonwhite_pop
    by_tract['race weight (pop)'] = np.divide(by_tract['race weight'],tract_pop_proportions)

    by_tract['node id'] = by_tract['location'].str.replace('Census Tract ', '')
    
    return by_tract